This notebook aims to measure the time saved by our modified scored unet architecture. You can find how to use the confidence score to save inference time by running the mask UNet less.

In [2]:
import os
import cv2
import sys
import glob
import copy
import time
import torch
import platform

import os.path as op
import numpy as np
from dotdict import dotdict
from matplotlib import pyplot as plt
import pathlib2
from pathlib2 import Path
from tqdm.notebook import tqdm, trange

sys.path.append('..')
from scripts.data import DataInterface
from scripts.model import ModelInteface, predict3d
from scripts.utils import *
from scripts.model.metrics import MPJPE

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# Get Basic Running Info
print('Current Directory: ', os.getcwd())
print('Current Node: ', platform.node())
print('Cuda availability: ', torch.cuda.is_available())

if platform.system() == 'Windows':
    temp = pathlib2.PosixPath
    pathlib2.PosixPath = pathlib2.WindowsPath

Current Directory:  e:\GitHub\dvs-hpe-light\notebook
Current Node:  Kirito
Cuda availability:  True


In [4]:
device = 'cuda'
frame_size = (260, 346)
mpjpe = MPJPE()
nidhp_root = r'L:\DVS\models\hpe\NiDHP'
midhp_root = r'L:\DVS\models\hpe\MiDHP'
mask_net_path = r'L:\DVS\models\unet\05-19-18-28-41\checkpoints\best-epoch=09-val_loss=0.021.ckpt'
model_date_dict = {
    'MiDHP': {
        'w_cl':'05-25-16-19-04',
        'const_time':'05-19-23-41-59',
        'occlusion': '10-17-21-15-12'
        },
    'NiDHP': {
        'w_cl':'06-01-16-13-56',
        'const_time':'06-01-16-17-07',
        'pretrain':'06-02-00-34-40'
        }
}

In [5]:
def time_string():
    """ Generate a time string from year to second.
    """
    return time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())

def get_model_path(date, root=midhp_root, best=True):
    model_subdir = op.join(root, date, 'checkpoints')
    if best:
        model_path = glob.glob(op.join(model_subdir, 'best*'))[0]
    else:
        model_path = op.join(model_subdir, 'last.ckpt')
    return model_path

def load_model(time_str='05-25-16-19-04', best=True, device='cuda', root=midhp_root):
    model_path = get_model_path(time_str, root=root, best=True)
    print(model_path)
    model = ModelInteface.load_from_checkpoint(model_path, mask_net_path=mask_net_path, detailed_test_record=True)
    model = model.to(device)
    print('Using trained model: ', model_path)
    return model

def manual_load_data(data_path, idx=0):
    reader = ToreSeqReader([data_path], labels_processed=False, percentile=90, redundant_labels=True)
    print("Total tore number:", reader.metas[0]['total_tore_count'])

    b_x, b_y = reader.load_seq_by_index(0, idx)
    tores = torch.tensor(b_x).float().to(device)
    del b_y['name']
    for k in b_y.keys():
        b_y[k] = torch.tensor(b_y[k])
    return b_x, b_y, tores

def inference(batch, model, verbose=False):
    b_x, b_y = batch
    del b_y['name']
    for k in b_y.keys():
        b_y[k] = torch.tensor(b_y[k])
    b_x = torch.tensor(b_x).float()
    normed_pred_joints, masks = model.predict_step(b_x.unsqueeze(0).to(device))
    normed_pred_joints = normed_pred_joints.cpu().detach()
    del batch

    if verbose:
        print('Label Keys: ', b_y.keys())
        print('Extrinsic:\n', b_y['M'][0].numpy().round(2))
        print('Intrinsic:\n', b_y['camera'][0].numpy().round(2))
    return normed_pred_joints, masks, b_x, b_y

suf = lambda n: "%d%s"%(n,{1:"st",2:"nd",3:"rd"}.get(n if n<20 else n%10,"th"))

In [6]:
device = 'cuda'
frame_size = (260, 346)
mpjpe = MPJPE()

# Spec Measure & Profiling

In [21]:
def get_gg_idx(l, thres):
    try:
        return next(i for i,v in enumerate(l) if v<thres)
    except:
        return len(l)
    
def denormalize_skeleton_batch(model, normed_pred_joints, b_y):
    pred_skcam = model.denormalize_predictions(normalized_predictions=normed_pred_joints, b_y=b_y)
    pred_skori = np.array([reproject_xyz_onto_world_coord(pred_skcam[i], b_y['M'][i]).numpy() for i in range(16)])
    return pred_skori

In [22]:
data_path = '/gypsum/scratch1/zhongyangzha/DVS_HPE/ntore_dataset/synthetic/Venti_left_nature_stream/meta.json'

human_name, camera_view = Path(data_path).parts[-2].split('_')[:2]
mask_root = f'/gypsum/work1/trahman/zhongyangzha/dvs_hpe/ntore_dataset/masks_acc/{human_name}/{camera_view}_alpha'

reader = ToreSeqReader([data_path], redundant_labels=True, labels_processed=True)
mask_reader = MaskSeqReader(mask_root)
batch = reader.get_pair_by_index(0,0)

model = load_model(time_str=model_date_dict['MiDHP']['w_cl'], best=True, device='cuda', root=midhp_root)

In [ ]:
thresholds = np.linspace(0.966,1,20)
total_time_records = []
mask_time_records = []
hpe_time_records = []
mpjpe_recorder = []
final_number = reader.metas[0]['total_tore_count'] - (reader.metas[0]['total_tore_count'] % 16)

for thres_idx, thres in enumerate(thresholds):
    tic1 = time.time()
    
    # Mask Prediction
    idx = 0
    masks_selected = []
    while True:
        batch = reader.get_pair_by_index(0, idx)
        tores, labels = batch

        tores = torch.tensor(tores[np.newaxis,...]).float().to(device)
        masks_pred, score_pred = model.model.mask_net(tores)
        masks_pred = torch.sigmoid(masks_pred)
        forward_num = max(1,get_gg_idx(score_pred.squeeze().cpu().detach(), thres))
        masks_selected.extend([*masks_pred.cpu().detach().numpy()[:forward_num]])
        idx += forward_num
        if idx >= final_number:
            masks_selected = masks_selected[:final_number]
            break
    
    tic2 = time.time()
    mask_time_records.append(tic2 - tic1)
    
    # HPE
    mpjpe_acc=[]
    for idx in range(final_number//16):
        batch = reader.load_seq_by_index(0, idx*16)
        tores, b_y = batch
        del b_y['name']
        for k in b_y.keys():
            b_y[k] = torch.tensor(b_y[k])
        tores = torch.tensor([tores]).float().to(device)
        mask_input = torch.tensor(np.stack(masks_selected[idx*16:(idx+1)*16])).unsqueeze(1).float().to(device)
        skeleton_ori = b_y['xyz']
        normed_pred_joints, _ = model.predict_step(tores, mask_input=mask_input)
        skeleton_pred = denormalize_skeleton_batch(model, normed_pred_joints.cpu().detach(), b_y)
        mpjpe_batch = np.mean([torch.mean(mpjpe(torch.tensor(skeleton_pred), skeleton_ori.cpu().detach())).item()])
        mpjpe_acc.append(mpjpe_batch)
    
    mpjpe_recorder.append(np.mean(mpjpe_acc))
    
    tic3 = time.time()
    hpe_time_records.append(tic3-tic2)
    total_time_records.append(tic3-tic1)
    print(f'No.{thres_idx}, Threshold: {round(thres,3)}, Mask Pred Time: {round(mask_time_records[-1], 3)}, HPE Time: {round(hpe_time_records[-1],3)}, Total Time: {round(total_time_records[-1],3)}, Mean MPJPE: {round(mpjpe_recorder[-1],3)}')

In [30]:
mega_info_pack = {
    'thresholds':thresholds,
    'total_time_records': total_time_records,
    'mask_time_records': mask_time_records,
    'hpe_time_records': hpe_time_records,
    'mpjpe_recorder': mpjpe_recorder
}
with open("./results/mega_info_pack_.yaml", 'w') as f:
    yaml.dump(mega_info_pack, f)

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

width=700
height=400
x = thresholds
mask_time_records = np.array(mask_time_records)
total_time_records = np.array(total_time_records)
mpjpe_recorder = np.array(mpjpe_recorder)
y1 = (1-mask_time_records/mask_time_records.max())*100
y2 = (1-total_time_records/total_time_records.max())*100
y3 = (1 - mpjpe_recorder/mpjpe_recorder.max())*100

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=thresholds,
    y=y1,
    name="Mask Time Saving Percentage"
))

fig.add_trace(go.Scatter(
    x=thresholds,
    y=y2,
    name="Total Time Saving Percentage"
))

fig.add_trace(go.Scatter(
    x=x,
    y=y3,
    name="Performance Rising Percentage"
))

fig.update_layout(width=width, height=height) #legend_title_text='Trend', 

fig.update_layout(
    xaxis_title="Confidence Score Threshold",
    yaxis_title="Percentage",
)

fig.show()
pio.write_image(fig, op.join('./results', "score-percentages.png"), width=width, height=height, scale=2)